In [30]:
from pandas_datareader import data as web
import urllib
import datetime as dt
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

In [44]:
start = dt.datetime(2019, 10, 1)
end = dt.datetime.today()

In [146]:
def preprocessing(df, prefix):
    """
    input: df - pandas dataframe; prefix - str
    output: df- pandas dataframe
    """
    df['change_rate'] = df['Close'].pct_change()
    df["price_change"] = df["Low"].diff()
    df = df.add_prefix(prefix + '_')
    
    return df

# sp500
sp500 = web.get_data_yahoo('^SP500TR',start, end)
sp500 = preprocessing(sp500, "sp500")

# usdjpy
usdjpy = web.DataReader("JPY=X", 'yahoo', start, end)
usdjpy  = preprocessing(usdjpy, "usdjpy")

# usdtwd
usdtwd = web.DataReader("TWD=X", 'yahoo', start, end)
usdtwd = preprocessing(usdtwd, "usdtwd")

#concat three dataframes
df = pd.concat([sp500, usdtwd, usdjpy], axis=1)

# as there is date difference in our raw data, we fillin the data to make asia/us date to the same date
target_list = ['usdtwd_High', 'usdtwd_Low', 'usdtwd_Open', 'usdtwd_Close',
       'usdtwd_Volume', 'usdtwd_Adj Close', 'usdtwd_change_rate',
       'usdtwd_price_change', 'usdjpy_High', 'usdjpy_Low', 'usdjpy_Open',
       'usdjpy_Close', 'usdjpy_Volume', 'usdjpy_Adj Close',
       'usdjpy_change_rate', 'usdjpy_price_change']

df[target_list] = df[target_list].fillna(method="bfill")


df = df[~df[['sp500_High']].isnull().all(1)]


In [149]:
df[["sp500_Low", "usdjpy_Low", "usdtwd_Low"]].corr()

,sp500_Low,usdjpy_Low,usdtwd_Low
sp500_Low,1.000000,0.671029,-0.713514
usdjpy_Low,0.671029,1.000000,-0.305362
usdtwd_Low,-0.713514,-0.305362,1.000000


In [170]:
fig = go.Figure(data=go.Scatter(x=df['sp500_Low'],
                                y=df['usdjpy_Low'],
                                mode='markers',
                                marker_color="red")) # hover text goes here

fig.update_layout(title='SP500_usdjpy')
fig.show()

In [171]:
fig = go.Figure(data=go.Scatter(x=df['sp500_Low'],
                                y=df['usdtwd_Low'],
                                mode='markers',
                                marker_color="red")) # hover text goes here

fig.update_layout(title='SP500_usdtwd')
fig.show()

In [150]:
df[["sp500_change_rate", "usdjpy_change_rate", "usdtwd_change_rate"]].corr()

,sp500_change_rate,usdjpy_change_rate,usdtwd_change_rate
sp500_change_rate,1.000000,-0.022899,-0.006143
usdjpy_change_rate,-0.022899,1.000000,-0.374723
usdtwd_change_rate,-0.006143,-0.374723,1.000000


In [151]:
df[["sp500_price_change", "usdjpy_price_change", "usdtwd_price_change"]].corr()

,sp500_price_change,usdjpy_price_change,usdtwd_price_change
sp500_price_change,1.000000,0.522768,-0.257865
usdjpy_price_change,0.522768,1.000000,-0.235519
usdtwd_price_change,-0.257865,-0.235519,1.000000


In [165]:

def plot_graph(df, col_name):

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index, y=df['sp500_' + col_name], name="sp500_"+ col_name, line_color='deepskyblue'))

    fig.add_trace(go.Scatter(
        x=df.index, y=df['usdtwd_' + col_name], name="usdtwd_" + col_name, line_color='dimgray', 
        yaxis="y2"
    ))

    fig.add_trace(go.Scatter(
        x=df.index, y=df['usdjpy_' + col_name], name="usdjpy_" + col_name, line_color='red',
        yaxis="y3"
    ))



    # Create axis objects
    fig.update_layout(
        yaxis=dict(
            tickfont=dict(color="deepskyblue")
        ),
        yaxis2=dict(
            tickfont=dict(color="dimgray"),
            anchor="free",
            overlaying="y",
            side="left"
        ),
        yaxis3=dict(
            tickfont=dict(color='red'),
            anchor="free",
            overlaying="y",
            side="left"
        ),

    width=1000,
    height=700,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    )



    fig.update_layout(title_text= col_name + ' Price Trend',
                      xaxis_rangeslider_visible=True)
    fig.show()


In [166]:
plot_graph(df, "Low")

In [167]:
plot_graph(df, "change_rate")

In [168]:
plot_graph(df, "price_change")